##### Copyright 2022 The IREE Authors

In [ ]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# IREE Cross Compilation Demo July 13, 2022

In [ ]:
#@title Setup

!python -m pip install iree-compiler iree-runtime iree-tools-tf -f https://github.com/iree-org/iree/releases/tag/candidate-20220713.203

import os
import tempfile
import tensorflow as tf

ARTIFACTS_DIR = os.path.join(tempfile.gettempdir(), "iree", "colab_artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
# %env IREE_SAVE_TEMPS=$ARTIFACTS_DIR/temps
print(f"Using artifacts directory '{ARTIFACTS_DIR}'")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://github.com/iree-org/iree/releases/tag/candidate-20220713.203
     |████████████████████████████████| 47.8 MB 12.2 MB/s 
     |████████████████████████████████| 2.2 MB 25.2 MB/s 
     |████████████████████████████████| 58.5 MB 25 kB/s 
Using artifacts directory '/tmp/iree/colab_artifacts'


In [ ]:
class DemoModule(tf.Module):
  # reduce_sum (dynamic input size, static output size)
  #   e.g. [1.0, 2.0, 3.0] -> 6.0
  @tf.function(input_signature=[tf.TensorSpec([None], tf.float32)])
  def reduce_sum(self, values):
    return tf.math.reduce_sum(values)

  # reduce_mean (dynamic input size, static output size)
  #   e.g. [1.0, 2.0, 3.0] -> 2.0
  @tf.function(input_signature=[tf.TensorSpec([None], tf.float32)])
  def reduce_mean(self, values):
    return tf.math.reduce_mean(values)

In [ ]:
#@title Import into MLIR

from IPython.display import clear_output
from iree.compiler import tf as tfc

compiler_module = tfc.compile_module(
    DemoModule(), import_only=True, output_mlir_debuginfo=False, output_generic_mlir=False)
clear_output()  # Skip over TensorFlow's output.
print("Demo MLIR (imported):\n```\n%s```\n" % compiler_module.decode("utf-8"))

Demo MLIR (imported):
```
"builtin.module"() ({
  "func.func"() ({
  ^bb0(%arg0: !iree_input.buffer_view):
    %0 = "iree_input.cast.buffer_view_to_tensor"(%arg0) : (!iree_input.buffer_view) -> tensor<?xf32>
    %1 = "func.call"(%0) {callee = @__inference_reduce_mean_70} : (tensor<?xf32>) -> tensor<f32>
    %2 = "iree_input.cast.tensor_to_buffer_view"(%1) : (tensor<f32>) -> !iree_input.buffer_view
    "func.return"(%2) : (!iree_input.buffer_view) -> ()
  }) {function_type = (!iree_input.buffer_view) -> !iree_input.buffer_view, iree.abi = "{\22a\22:[[\22ndarray\22,\22f32\22,1,null]],\22r\22:[[\22ndarray\22,\22f32\22,0]],\22v\22:1}", sym_name = "reduce_mean"} : () -> ()
  "func.func"() ({
  ^bb0(%arg0: tensor<?xf32>):
    %0 = "arith.constant"() {value = 0 : index} : () -> index
    %1 = "mhlo.constant"() {value = dense<-0.000000e+00> : tensor<f32>} : () -> tensor<f32>
    %2 = "mhlo.reduce"(%arg0, %1) ({
    ^bb0(%arg1: tensor<f32>, %arg2: tensor<f32>):
      %8 = "mhlo.add"(%arg1, %arg

In [ ]:
#@title Compile to IREE .vmfb files

from iree.compiler import compile_str

vmvx_blob = compile_str(
    compiler_module,
    input_type="mhlo",
    target_backends=["vmvx"],
    extra_args=["--mlir-print-ir-after=iree-mhlo-to-linalg-on-tensors"])
vmvx_path = os.path.join(ARTIFACTS_DIR, "demo_vmvx.vmfb")
with open(vmvx_path, "wb") as output_file:
  output_file.write(vmvx_blob)
print(f"Wrote .vmfb to path '{vmvx_path}'")

wasm_blob = compile_str(
    compiler_module,
    input_type="mhlo",
    target_backends=["llvm"],
    extra_args=["--iree-llvm-target-triple=wasm32-unknown-emscripten"])
wasm_path = os.path.join(ARTIFACTS_DIR, "demo_wasm.vmfb")
with open(wasm_path, "wb") as output_file:
  output_file.write(wasm_blob)
print(f"Wrote .vmfb to path '{wasm_path}'")

android_cpu_blob = compile_str(
    compiler_module,
    input_type="mhlo",
    target_backends=["llvm"],
    extra_args=["--iree-llvm-target-triple=aarch64-none-linux-android29"])
android_cpu_path = os.path.join(ARTIFACTS_DIR, "demo_android-cpu-arm64-v8a.vmfb")
with open(android_cpu_path, "wb") as output_file:
  output_file.write(android_cpu_blob)
print(f"Wrote .vmfb to path '{android_cpu_path}'")

android_gpu_blob = compile_str(
    compiler_module,
    input_type="mhlo",
    target_backends=["vulkan-spirv"],
    extra_args=["--iree-vulkan-target-triple=valhall-unknown-android11"])
android_gpu_path = os.path.join(ARTIFACTS_DIR, "demo_android-gpu-mali.vmfb")
with open(android_gpu_path, "wb") as output_file:
  output_file.write(android_gpu_blob)
print(f"Wrote .vmfb to path '{android_gpu_path}'")

// -----// IR Dump After ConvertMHLOToLinalgOnTensors //----- //
func.func @reduce_mean(%arg0: !iree_input.buffer_view loc("<stdin>":3:8)) -> !iree_input.buffer_view attributes {iree.abi = "{\22a\22:[[\22ndarray\22,\22f32\22,1,null]],\22r\22:[[\22ndarray\22,\22f32\22,0]],\22v\22:1}"} {
  %0 = iree_input.cast.buffer_view_to_tensor %arg0 : !iree_input.buffer_view -> tensor<?xf32> loc("<stdin>":4:10)
  %1 = call @__inference_reduce_mean_70(%0) : (tensor<?xf32>) -> tensor<f32> loc("<stdin>":5:10)
  %2 = iree_input.cast.tensor_to_buffer_view %1 : tensor<f32> -> !iree_input.buffer_view loc("<stdin>":6:10)
  return %2 : !iree_input.buffer_view loc("<stdin>":7:5)
} loc("<stdin>":2:3)

// -----// IR Dump After ConvertMHLOToLinalgOnTensors //----- //
func.func private @__inference_reduce_mean_70(%arg0: tensor<?xf32> {tf._user_specified_name = "values"} loc("<stdin>":10:8)) -> tensor<f32> attributes {tf._construction_context = "kEagerRuntime", tf._input_shapes = [#tf_type.shape<?>]} {
  %c0 = ari

Wrote .vmfb to path '/tmp/iree/colab_artifacts/demo_vmvx.vmfb'
Wrote .vmfb to path '/tmp/iree/colab_artifacts/demo_wasm.vmfb'
Wrote .vmfb to path '/tmp/iree/colab_artifacts/demo_android-cpu-arm64-v8a.vmfb'
Wrote .vmfb to path '/tmp/iree/colab_artifacts/demo_android-gpu-mali.vmfb'


In [ ]:
#@title Download artifacts

ARTIFACTS_ZIP = "/tmp/iree_demo_colab_artifacts.zip"

print(f"Zipping '{ARTIFACTS_DIR}' to '{ARTIFACTS_ZIP}' for download...")
!cd {ARTIFACTS_DIR} && zip -r {ARTIFACTS_ZIP} .

# Note: you can also download files using Colab's file explorer
try:
  from google.colab import files
  print("Downloading the artifacts zip file...")
  files.download(ARTIFACTS_ZIP)
except ImportError:
  print("Missing google_colab Python package, can't download files")

Zipping '/tmp/iree/colab_artifacts' to '/tmp/iree_demo_colab_artifacts.zip' for download...
  adding: demo_android-cpu-arm64-v8a.vmfb (deflated 65%)
  adding: demo_wasm.vmfb (deflated 58%)
  adding: demo_android-gpu-mali.vmfb (deflated 64%)
  adding: demo_vmvx.vmfb (deflated 61%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>